In [8]:
import pandas as pd
import glob
import os

def load_csv_files_from_folder(folder_path):

    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    loaded_dataframes = []

    if not csv_files:
        # Minimal report: only if no files are found at all
        print(f"No CSV files found in '{folder_path}'.")
        return loaded_dataframes

    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            loaded_dataframes.append(df)
        except Exception:
            # Minimal error handling: simply skip the file and do not report details
            pass # No action/report as requested ("just do only that")

    return loaded_dataframes

# --- Execution ---
yfinance_data_folder = '../data/yfinance_data/' 

# Load all CSVs into a list of DataFrames
all_yfinance_dfs = load_csv_files_from_folder(yfinance_data_folder)


print(f"Successfully loaded {len(all_yfinance_dfs)} DataFrames.")
if all_yfinance_dfs:
    print("\nHead of the first loaded DataFrame:")
    print(all_yfinance_dfs[0].head())

Successfully loaded 7 DataFrames.

Head of the first loaded DataFrame:
         Date      Open      High       Low     Close  Adj Close      Volume  \
0  1986-03-13  0.088542  0.101563  0.088542  0.097222   0.059946  1031788800   
1  1986-03-14  0.097222  0.102431  0.097222  0.100694   0.062087   308160000   
2  1986-03-17  0.100694  0.103299  0.100694  0.102431   0.063158   133171200   
3  1986-03-18  0.102431  0.103299  0.098958  0.099826   0.061552    67766400   
4  1986-03-19  0.099826  0.100694  0.097222  0.098090   0.060482    47894400   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [13]:
import pandas as pd

# Define ALL the required columns for each DataFrame
REQUIRED_COLUMNS = [
    'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close',
    'Volume', 'Dividends', 'Stock Splits'
]

# Define a mapping for common alternative column names to standard names
COLUMN_NAME_MAPPING = {
    'date': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'adj close': 'Adj Close',
    'adj_close': 'Adj Close',
    'volume': 'Volume',
    'vol': 'Volume',
    'dividends': 'Dividends',
    'dividend': 'Dividends',
    'stock splits': 'Stock Splits',
    'stock_splits': 'Stock Splits'
}

# --- Function to process each DataFrame ---
def standardize_df_columns(df, required_cols, col_mapping):

    # 1. Standardize column names (lowercase original names first for mapping)
    df.columns = df.columns.str.lower()
    df.rename(columns=col_mapping, inplace=True)

    # 2. Add missing required columns with pd.NA
    for col in required_cols:
        if col not in df.columns:
            df[col] = pd.NA

    # 3. Select ONLY the required columns and reorder them
    # This automatically drops any columns not in `required_cols`
    df = df[required_cols]

    return df

# --- Process all loaded DataFrames ---
# This list will store your DataFrames after column standardization
processed_yfinance_dfs = []

# Loop through each DataFrame loaded in the previous step
for df in all_yfinance_dfs:
    try:
        # Process the DataFrame
        processed_df = standardize_df_columns(df.copy(), REQUIRED_COLUMNS, COLUMN_NAME_MAPPING)
        processed_yfinance_dfs.append(processed_df)
    except Exception:
        pass 

print(f"Finished processing {len(processed_yfinance_dfs)} DataFrames.")
if processed_yfinance_dfs:
    print("\nExample: Head of the first processed DataFrame:")
    print(processed_yfinance_dfs[0].head())
    print("\nExample: Columns of the first processed DataFrame:")
    print(processed_yfinance_dfs[0].columns.tolist())
    print("\nExample: Check shape and columns of all processed DFs:")
    for i, df in enumerate(processed_yfinance_dfs):
        print(f"DF {i+1}: Shape {df.shape}, Columns {df.columns.tolist()}")

Finished processing 7 DataFrames.

Example: Head of the first processed DataFrame:
            Date      Open      High       Low     Close  Adj Close  \
Date                                                                  
1986-03-13  <NA>  0.088542  0.101563  0.088542  0.097222   0.059946   
1986-03-14  <NA>  0.097222  0.102431  0.097222  0.100694   0.062087   
1986-03-17  <NA>  0.100694  0.103299  0.100694  0.102431   0.063158   
1986-03-18  <NA>  0.102431  0.103299  0.098958  0.099826   0.061552   
1986-03-19  <NA>  0.099826  0.100694  0.097222  0.098090   0.060482   

                Volume  Dividends  Stock Splits  
Date                                             
1986-03-13  1031788800        0.0           0.0  
1986-03-14   308160000        0.0           0.0  
1986-03-17   133171200        0.0           0.0  
1986-03-18    67766400        0.0           0.0  
1986-03-19    47894400        0.0           0.0  

Example: Columns of the first processed DataFrame:
['Date', 'Open', 

In [14]:
import pandas as pd
import talib as ta
import numpy as np

# Important Pre-computation Check (if not already done in previous steps) 
# Ensure 'Date' is set as index and sorted for all DFs, and OHLCV columns are numeric.
# This loop ensures the data is in the correct format for TA-Lib.
for i, df_stock in enumerate(all_yfinance_dfs):
    # Ensure 'Date' is datetime and set as index if it's still a column
    if 'Date' in df_stock.columns:
        df_stock['Date'] = pd.to_datetime(df_stock['Date'], errors='coerce')
        df_stock.dropna(subset=['Date'], inplace=True) # Remove rows where Date conversion failed
        df_stock.set_index('Date', inplace=True)
    
    # Crucial: Sort by index (Date) to ensure correct time-series calculation for TA-Lib
    df_stock.sort_index(inplace=True)

    # Ensure OHLCV columns are numeric (float) and handle potential NaNs
    numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
    for col in numeric_cols:
        if col in df_stock.columns:
            df_stock[col] = pd.to_numeric(df_stock[col], errors='coerce')
    df_stock.dropna(subset=numeric_cols, inplace=True) # Drop rows if core OHLCV values are missing after conversion


print("--- Applying SMA, RSI, and MACD Indicators using TA-Lib to each DataFrame ---")

# Loop through each DataFrame in your list of 7 datasets
for i, df_stock in enumerate(all_yfinance_dfs):
    ticker_name = df_stock['Ticker'].iloc[0] if 'Ticker' in df_stock.columns else f'DF_{i+1}'
    print(f"\nProcessing indicators for {ticker_name} (Original Shape: {df_stock.shape})...")

    # Extract necessary data for TA-Lib as NumPy arrays 
    close_prices = df_stock['Close'].values.astype(float)
   
    # Minimum data points required for MACD (which has the longest lookback here)
    min_data_points_required = 35 
    if len(df_stock) < min_data_points_required:
        print(f"  Warning: Not enough data points ({len(df_stock)}) in {ticker_name} for some indicators. Skipping TA-Lib calculations for this DataFrame.")
        continue 

    try:
        # 1. Simple Moving Average (SMA) - using a 20-period
        df_stock['SMA_20'] = ta.SMA(close_prices, timeperiod=20)

        # 2. Relative Strength Index (RSI) - using a 14-period
        df_stock['RSI_14'] = ta.RSI(close_prices, timeperiod=14)

        # 3. Moving Average Convergence Divergence (MACD)
        # Returns three arrays: MACD line, Signal line, and MACD Histogram
        macd, macdsignal, macdhist = ta.MACD(
            close_prices,
            fastperiod=12,
            slowperiod=26,
            signalperiod=9
        )
        df_stock['MACD'] = macd
        df_stock['MACD_Signal'] = macdsignal
        df_stock['MACD_Hist'] = macdhist

        print(f"  Successfully added SMA, RSI, and MACD to {ticker_name}. New shape: {df_stock.shape}")

    except Exception as e:
        print(f"  Error applying TA-Lib indicators to {ticker_name}: {e}")

# --- Verification: Display a sample of one of the modified DataFrames ---
print("\n--- Example of the first DataFrame (DF 1) after adding indicators ---")
if all_yfinance_dfs:
    print(all_yfinance_dfs[0].tail())
    print(f"\nNew columns added to DF 0: {all_yfinance_dfs[0].columns.tolist()}")
else:
    print("No DataFrames were processed. Please ensure 'all_yfinance_dfs' is populated.")

--- Applying SMA, RSI, and MACD Indicators using TA-Lib to each DataFrame ---

Processing indicators for DF_1 (Original Shape: (9672, 13))...
  Successfully added SMA, RSI, and MACD to DF_1. New shape: (9672, 13)

Processing indicators for DF_2 (Original Shape: (6421, 13))...
  Successfully added SMA, RSI, and MACD to DF_2. New shape: (6421, 13)

Processing indicators for DF_3 (Original Shape: (6846, 13))...
  Successfully added SMA, RSI, and MACD to DF_3. New shape: (6846, 13)

Processing indicators for DF_4 (Original Shape: (3545, 13))...
  Successfully added SMA, RSI, and MACD to DF_4. New shape: (3545, 13)

Processing indicators for DF_5 (Original Shape: (2926, 13))...
  Successfully added SMA, RSI, and MACD to DF_5. New shape: (2926, 13)

Processing indicators for DF_6 (Original Shape: (10998, 13))...
  Successfully added SMA, RSI, and MACD to DF_6. New shape: (10998, 13)

Processing indicators for DF_7 (Original Shape: (5020, 13))...
  Successfully added SMA, RSI, and MACD to DF_